In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from ae import AutoEncoder
import torch
from skimage.metrics import structural_similarity as ssim
from torchvision.transforms import transforms
%reload_ext autoreload
%autoreload 2


In [3]:
def load_image_as_np(image_path):
    return np.asarray(Image.open(image_path).convert('RGB'), dtype=np.uint8)

def store_image_from_np(image_path,data,format='RGB'):
    img = Image.fromarray(data, format)
    img.save(image_path)
    return img

In [4]:
def segment_image(image,tile_size,pad_type='reflect'):
    img_height,img_width = image.shape[:2]

    # Pads the image so it can be chunked down to a grid even if the size of the image is not
    # divisible by the chunk size
    v_pad = (0,tile_size - (img_height % tile_size)) if img_height % tile_size != 0 else (0,0)
    h_pad = (0,tile_size - (img_width % tile_size)) if img_width % tile_size != 0 else (0,0)
        
    image = np.pad(image, (v_pad,h_pad,(0,0)), pad_type)

    img_height , img_width, channels = image.shape

    tiled_array =  image.reshape(img_height // tile_size,
                                 tile_size,
                                 img_width // tile_size,
                                 tile_size,
                                 channels)

    tiled_array = tiled_array.swapaxes(1,2)

    return np.concatenate(tiled_array,axis=0)


In [5]:
def rebuild_image(tile_array,image_size,tile_size):
    img_height, img_width, channels = image_size
    
    tile_rows = int(np.ceil(img_height/tile_size))
    tile_cols = int(np.ceil(img_width/tile_size))

    tile_array = tile_array.reshape(tile_rows,
                                    tile_cols,
                                    tile_size,
                                    tile_size,
                                    channels)
    
    tile_array = np.concatenate(tile_array,axis=1)
    tile_array = np.concatenate(tile_array,axis=1)

    return tile_array[:img_height,:img_width]

In [6]:
def get_model(model_path):
    model = AutoEncoder.load_autoencoder(model_path)
    return send_to_device(model)


def send_to_device(pytorch_object):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pytorch_object = pytorch_object.to(device)
    return pytorch_object


def apply_compress_function(tile_tensor, model_path):
    model = get_model(model_path)
    return model.encode(send_to_device(tile_tensor))

def apply_decompression_function(encoded_tile_tensor,model_path) :
    model = get_model(model_path)
    return model.decode(encoded_tile_tensor)

In [7]:
def make_tensor(tile_list_array:np.array)-> torch.tensor:    
    tile_list_array = tile_list_array.swapaxes(2,3)
    tile_list_array = tile_list_array.swapaxes(1,2)
    
    tile_list_tensor = torch.from_numpy(tile_list_array.astype('float32')).reshape(-1,3*8*8)
    #print("Make Tensor tile_list_array.shape=",tile_list_tensor.size())
    return tile_list_tensor /255


def retrieve_array(decoded_tile_tensor):
    res = decoded_tile_tensor.detach().cpu().reshape(-1,3,8,8)
    res = (res.numpy() * 255).astype('uint8')
    res = res.swapaxes(1,2)    
    res = res.swapaxes(2,3)
    return res

In [ ]:
wd_path = os.path.abspath(os.getcwd())
models_path = os.path.join(wd_path, "models")
model_used_path = os.path.join(models_path, "ae_1.pt")
image_path = os.path.join(wd_path,"images","val_0_0.jpeg")
image_path_out = os.path.join(wd_path,"images","val_0_0_out.jpeg")
tile_size= 8

test_image                  = load_image_as_np(image_path)
print("Test Image          ",test_image.shape)

tile_list                   = segment_image(test_image,tile_size=8,pad_type='reflect')
print("Tile List           ",tile_list.shape)

tile_tensor                 = make_tensor(tile_list)
print("Tile_tensor torch   ",tile_tensor.size())

compressed_image_tensor     = apply_compress_function(tile_tensor,model_used_path)
print("Comp Image Tensor   ",compressed_image_tensor)

#-------------------- DECOMPRESSION --------------
decompressed_image_tensor   = apply_decompression_function(compressed_image_tensor,model_used_path)
print("Rebuilt Image Tensor",decompressed_image_tensor.size())

decompresssed_image         = retrieve_array(decompressed_image_tensor)
print("Rebuilt image array ",decompresssed_image.shape)

end_image                   = rebuild_image(decompresssed_image,test_image.shape,tile_size)
print("Rebuilt Image       ",end_image.shape)

store_image_from_np(image_path_out, end_image)
t = transforms.ToTensor()
print("="*40)

print("Reconstructed")

plt.imshow(end_image)
plt.axis('off')
plt.show()
print("="*40)
print("Original")

plt.imshow(test_image)

plt.axis('off')
plt.show()

In [53]:
wd_path = os.path.abspath(os.getcwd())
model_path = os.path.join(wd_path, "models","ae10_0.pt")



In [3]:
def get_mse(original_image,retrieved_image):
    #Mean Square Error
    diff = original_image-retrieved_image
    return (np.square(diff)).mean(axis=None)

In [4]:
def get_psnr(original_image,retrieved_image):
    #Peak signal to noise ratio
    mse = get_mse(original_image,retrieved_image)
    max = 255
    return 20*np.log10(max)-10*np.log10(mse)

In [82]:
from autoencoder_compressor import Compressor_Decompressor
wd_path = os.path.abspath(os.getcwd())
model_path = os.path.join(wd_path, "models","ae10_0.pt")
image_path = os.path.join(wd_path,"images","testing","misc","4.2.03.tiff")
image_path_out = os.path.join(wd_path,"images","4.2.03_out.tiff")

comp = Compressor_Decompressor(model_path,compression_out=10)
c , image_size = comp.compress_image(image_path)

res = comp.decompress_image(c,image_path_out,image_size,return_image=True)

original = Compressor_Decompressor.load_image_as_np(image_path)
print(original.nbytes)
print(np.asarray(c).nbytes)
print("Ratio = ", original.nbytes/np.asarray(c).nbytes)
print("MSE :" ,get_mse(original,res))
print("PSNR:" ,get_psnr(original,res))
print("SSIM:" ,ssim(original,res,channel_axis=2))



786432
81920
Ratio =  9.6
MSE : 85.55173238118489
PSNR: 28.80851552648633
SSIM: 0.48653362769103553
